# Scenario 1

In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
def read_logs(experiment:int):
    tx_logs = pd.read_csv(f"../logs/s1_v{experiment}_transmitter.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s1_v{experiment}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    tx_logs["EXPERIMENT"] = [experiment]*tx_logs.shape[0]
    rx_logs["EXPERIMENT"] = [experiment]*rx_logs.shape[0]
    
    return pd.concat([tx_logs, rx_logs], ignore_index=True)

In [3]:
def read_experiments():
    experiments = pd.read_csv("../logs/s1_experiments.csv", sep=';')
    
    experiments["VELOCITY"] = (2*np.pi*experiments["RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"]
    
    return experiments

## Experiments

In [4]:
logs = pd.DataFrame([], columns=['SEQ', 'ACK', 'TIMESTAMP', 'PAYLOAD', 'RSSI', 'LQI', 'ROLE', 'EXPERIMENT'])

for experiment in range(1, 7):
    logs = pd.concat([logs, read_logs(experiment)], ignore_index=True)

tx_logs = logs[logs["ROLE"]=="Tx"]
rx_logs = logs[logs["ROLE"]=="Rx"]

experiments = read_experiments()

logs_exps = pd.merge(logs, experiments, on="EXPERIMENT")

### Round Trip Times (RTT)

In [5]:
RTTs = pd.DataFrame((tx_logs[tx_logs["ACK"]==240].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"] - tx_logs[tx_logs["ACK"]==0].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"]))
RTTs.columns=["RTT"]
RTTs = RTTs.reset_index()

In [6]:
alt.Chart(RTTs, title="Round Trip Times in Scenario 1").mark_bar().encode(
    x="SEQ:O",
    y=alt.Y("RTT:Q", title="RTT (ms)"),
    row="EXPERIMENT:N"
)

alt.Chart(...)

### RSSI

In [7]:
RSSI_df = logs.loc[((logs["ACK"]==0) & (logs["ROLE"]=="Rx")) | ((logs["ACK"]==240) & (logs["ROLE"]=="Tx"))][["SEQ", "RSSI", "ROLE", "EXPERIMENT"]].reset_index()

In [8]:
alt.Chart(RSSI_df, title=f"RSSI Values in Scenario 1 Experiment {experiment}").mark_point(filled=True).encode(
    x="SEQ:O",
    y=alt.Y("RSSI:Q", scale=alt.Scale(domain=(min(RSSI_df["RSSI"])-10, max(RSSI_df["RSSI"])+10)), title = "RSSI (dBm)"),
    color="ROLE:N",
    row="EXPERIMENT:N"
)

alt.Chart(...)

In [9]:
alt.Chart(RSSI_df, title=f"RSSI Average Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y=alt.Y("average(RSSI)", title = "average RSSI (dBm)"),
    color="ROLE:N",
    column="EXPERIMENT:N"
)

alt.Chart(...)

In [10]:
alt.Chart(RSSI_df, title=f"RSSI Variances Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="variance(RSSI)",
    color="ROLE:N",
    column="EXPERIMENT:N"
)

alt.Chart(...)

### LQI

In [11]:
LQI_df = logs.loc[((logs["ACK"]==0) & (logs["ROLE"]=="Rx")) | ((logs["ACK"]==240) & (logs["ROLE"]=="Tx"))][["SEQ", "LQI", "ROLE", "EXPERIMENT"]]

alt.Chart(LQI_df, title=f"LQI Values in Scenario 1 Experiment {experiment}").mark_point(filled=True).encode(
    x="SEQ:O",
    y=alt.Y("LQI:Q", scale=alt.Scale(domain=(min(LQI_df["LQI"])-5, max(LQI_df["LQI"])+5))),
    color="ROLE:N",
    row="EXPERIMENT:N"
)

alt.Chart(...)

In [12]:
alt.Chart(LQI_df, title=f"LQI Average Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="average(LQI)",
    color="ROLE:N",
    column="EXPERIMENT:N"
)

alt.Chart(...)

In [13]:
alt.Chart(LQI_df, title=f"LQI Variances Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="variance(LQI)",
    color="ROLE:N",
    column="EXPERIMENT:N"
)

alt.Chart(...)

## Comparison

In [114]:
title_of = {
    "RTT": "RTT (ms)",
    "RSSI": "RSSI (dBm)",
    "LQI": "LQI",
    "ROLE": "Role",
    "MOVING_ROLE": "Moving Role",
    "PAYLOAD_LENGTH": "PLL",
    "VELOCITY": "Velocity (m/s)",
    "RADIUS_M": "Radius (m)"
}

long_title_of = {
    "RTT": "Round Trip Time",
    "RSSI": "Received Signal Strength Indicator",
    "LQI": "Link Quality Indicator",
    "PAYLOAD_LENGTH": "Payload Length"
}

In [115]:
def make_average_chart(data:pd.DataFrame, subject:str, column:str):
    return alt.Chart(data, title=long_title_of[subject]).mark_bar().encode(
        x=alt.X(f"{column}:O", title=title_of[column]),
        y=alt.Y(f"average({subject}):Q", title=f"average {title_of[subject]}"),
        color=alt.Color(f"{column}:N", title=title_of[column])
    )

def average_chart(one_exp:int, other_exp:int, data:pd.DataFrame, subject:str, column:str):
    data_PL = data.loc[(data["EXPERIMENT"] == one_exp) | (data["EXPERIMENT"] == other_exp)]
    data_PL = pd.merge(data_PL, experiments, on="EXPERIMENT")
    return make_average_chart(data_PL, subject, column)

def compare_two_experiments_average_metrics(one_exp:int, other_exp:int, column:str, plot_title:str):
    RTTs_Chart = average_chart(one_exp, other_exp, RTTs, "RTT", column)
    RSSI_Chart = average_chart(one_exp, other_exp, RSSI_df, "RSSI", column)
    LQI_Chart = average_chart(one_exp, other_exp, LQI_df, "LQI", column)

    return alt.hconcat(
        RTTs_Chart, RSSI_Chart, LQI_Chart,
        title = plot_title
    )

### Moving Role

In [66]:
compare_two_experiments_average_metrics(1, 2, "MOVING_ROLE", "Comparison On Rx or Tx Moving (Radius 10 m)")

alt.HConcatChart(...)

### Payload Length

In [67]:
compare_two_experiments_average_metrics(2, 3, "PAYLOAD_LENGTH", "Comparison Upon Different Payload Lengths (Radius 10 m)")

alt.HConcatChart(...)

In [68]:
compare_two_experiments_average_metrics(5, 6, "PAYLOAD_LENGTH", "Comparison Upon Different Payload Lengths (Radius 40 m)")

alt.HConcatChart(...)

### Velocity

In [69]:
compare_two_experiments_average_metrics(3, 4, "VELOCITY", "Comparison Upon Velocity (Radius 10 m)")

alt.HConcatChart(...)

### Radius

In [70]:
compare_two_experiments_average_metrics(3, 5, "RADIUS_M", "Comparison Upon Radius")

alt.HConcatChart(...)

In [71]:
compare_two_experiments_average_metrics(2, 6, "RADIUS_M", "Comparison Upon Radius")

alt.HConcatChart(...)

# Evaluation

## Payload Length

In [216]:
from pandas.api.types import is_numeric_dtype

def make_average_chart(data:pd.DataFrame, subject:str, column:str, facet_column:str=None, digits=0):
    base = alt.Chart(data, title=long_title_of[subject]).encode(
        x=alt.X(f"{column}:O", title=title_of[column], axis=alt.Axis() if not is_numeric_dtype(data[column]) else alt.Axis(format=f',.{digits}f')),
        y=alt.Y(f"average({subject}):Q", title=f"average {title_of[subject]}"),
        color=alt.Color(f"{column}:N", title=title_of[column], legend=None),
        column=alt.Column(facet_column, title=title_of[facet_column]) if facet_column is not None else alt.Column()
    )
    return base.mark_bar()

def average_chart(exps, data:pd.DataFrame, subject:str, column:str, facet_column:str=None, digits=0):
    data_PL = data.loc[data["EXPERIMENT"].isin(exps)]
    data_PL = pd.merge(data_PL, experiments, on="EXPERIMENT")
    return make_average_chart(data_PL, subject, column, facet_column, digits)

def compare_two_experiments_average_metrics(exps, column:str, facet_column:str=None, plot_title:str="", digits=0, metrics=["RTT", "RSSI", "LQI"]):
    charts = []
    
    if "RTT" in metrics:
        charts.append(average_chart(exps, RTTs, "RTT", column, facet_column, digits))
        
    if "RSSI" in metrics:
        charts.append(average_chart(exps, RSSI_df, "RSSI", column, facet_column, digits))
        
    if "LQI" in metrics:
        
    LQI_Chart = average_chart(exps, LQI_df, "LQI", column, facet_column, digits)

    return alt.hconcat(
        *charts,
        title = plot_title + " — Experiments " + ", ".join([str(i) for i in exps])
    )

In [217]:
compare_two_experiments_average_metrics([2, 3, 5, 6], "PAYLOAD_LENGTH", "RADIUS_M", f"Comparison Upon Different Payload Lengths ({title_of['PAYLOAD_LENGTH']})")

alt.HConcatChart(...)

## Moving Role

In [206]:
compare_two_experiments_average_metrics([1, 2], "MOVING_ROLE", plot_title=f"Comparison Upon Moving Role")

alt.HConcatChart(...)

## Radius

In [207]:
compare_two_experiments_average_metrics([3, 5, 2, 6], "RADIUS_M", "PAYLOAD_LENGTH", plot_title=f"Comparison Upon Radius")

alt.HConcatChart(...)

## Velocity

In [208]:
compare_two_experiments_average_metrics([3, 4], "VELOCITY", plot_title=f"Comparison Upon Velocity", digits=1)

alt.HConcatChart(...)